# Heroes Of Pymoli - Data Analysis
    
    Of the 576 unique players an overwhelming majority are males (84%). Over 76% of players are between the ages of 15 and 29. Men (652) made substantially more purchases than women (113), but, women tended to spend about 10% more on average per purchase. ($4.47 vs. $4.07). The items purchases were varied. The 5 most frequently purchased items only make up 7% of the total revenue generated.

In [220]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Create a variable that holds the path to the CSV file (I will put the csv file in the same directory as the .pynb file)
file_to_load = "purchase_data.csv"

# Load the CSV file and show the first five entries
purchase_data = pd.read_csv(file_to_load)

# print the first 5 rows of the initial data frame
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [235]:
# Create a data frame using the .describe() function on the original data to view quick stats on the full sample
OriginalStats_df = purchase_data.describe()

OriginalStats_df

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,92.114103,3.050987
std,225.310896,6.659444,52.775943,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,48.000000,1.980000
50%,389.500000,22.000000,93.000000,3.150000
75%,584.250000,25.000000,139.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [222]:
# Find the number of unique users using the screen name

UniqueUsers = len(purchase_data["SN"].unique())
UniqueUsers_df = pd.DataFrame({"Player Count": [UniqueUsers]})
UniqueUsers_df

,Player Count
0,576


In [244]:
### Purchasing Analysis (Total)
# Number of Unique Items, Average Purchase Price, Total Number of Purchases, & Total Revenue

# Create variables to calculate the purchase information I am looking for
unique_items = len(purchase_data["Item ID"].unique())
avg_price = purchase_data["Price"].mean()
ttl_purchases = purchase_data["Purchase ID"].sum()
ttl_revenue = purchase_data["Price"].sum()

# Create a data frame using those variables
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                                    "Average Price": [avg_price],
                                    "Total Number of Purchases": [ttl_purchases],
                                    "Total Revenue": [ttl_revenue]}) 

# Change AVG Price and Total revenue to $s
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${:.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)

# print the new data frame
purchasing_analysis

,Number of Unique Items,Average Price,Total Number of Purchases,Total Revenue
0,183,$3.05,303810,$2379.77


In [245]:
### Gender Demographics
# Percentage and Count of Male Players, Percentage and Count of Female Players, & Percentage and Count of Other / Non-Disclosed

# Eliminate duplicate screen names from the data set 
no_duplicates = purchase_data.drop_duplicates(subset='SN', keep="first")

# Count the instances of each gender
men_count = no_duplicates["Gender"].value_counts()['Male']
women_count = no_duplicates["Gender"].value_counts()['Female']
other_count = no_duplicates["Gender"].value_counts()['Other / Non-Disclosed']

# Calculate the percentages
men_percent = ((men_count/(men_count+women_count+other_count))*100)
women_percent = (women_count/(men_count+women_count+other_count))*100
other_percent = (other_count/(men_count+women_count+other_count))*100

# Create the required data frame
gender_count_df = pd.DataFrame({"Gender": ['Male', 'Female', 'Other/ Non-disclosed'],
                               "Total": [men_count, women_count, other_count],
                               "Percentage of Players": [men_percent, women_percent, other_percent]})

# format the % of players using mapping
gender_count_df["Percentage of Players"] = gender_count_df["Percentage of Players"].map("{:.2f}%".format)

# Print the data frame
gender_count_df

,Gender,Total,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other/ Non-disclosed,11,1.91%


In [238]:
### Purchasing Analysis (Gender)

# The below each broken by gender
#  * Purchase Count
#  * Average Purchase Price
#  * Total Purchase Value
#  * Average Purchase Total per Person by Gender

# Create groupby objects by gender (one for all purchase instances and one without duplicated users)
groupby_gender_dupes = purchase_data.groupby(['Gender'])
print(groupby_gender_dupes)

groupby_gender_nodupes = no_duplicates.groupby(['Gender'])
print(groupby_gender_nodupes)

# Calculate all the new values using a groupby function
purchase_count = groupby_gender_dupes["Item Name"].count()
gender_count_new = groupby_gender_nodupes["Gender"].count()
avg_price = groupby_gender_dupes["Price"].mean()
ttl_price = groupby_gender_dupes["Price"].sum()
ttl_avg_per = groupby_gender_nodupes["Price"].count()
ttl_cost_per_person = ttl_price/ttl_avg_per

# Reset the index on the groupby objects so they can be placed into a dataframe
purchase_count.reset_index(0)
avg_price.reset_index(0)
ttl_price.reset_index(0)
ttl_avg_per.reset_index(0)
ttl_cost_per_person.reset_index(0)

# Create the required data frame
purchase_by_gender = pd.DataFrame({"Purchase Count": purchase_count,
                               "Average Purchase Price": avg_price,
                               "Total Purchase Value": ttl_price,
                               "Average Purchase Total per Person": ttl_cost_per_person})


# Reformat the columns containing $ values
purchase_by_gender["Average Purchase Price"] = purchase_by_gender["Average Purchase Price"].map("${:.2f}".format)
purchase_by_gender["Total Purchase Value"] = purchase_by_gender["Total Purchase Value"].map("${:.2f}".format)
purchase_by_gender["Average Purchase Total per Person"] = purchase_by_gender["Average Purchase Total per Person"].map("${:.2f}".format)

# Return the dataframe
purchase_by_gender.head()


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [226]:
### Age Demographics

# The below each broken into bins of 4 years (i.e. 0-9, 10-14, 15-19, etc.)
#  * Purchase Count
#  * Average Purchase Price
#  * Total Purchase Value
#  * Average Purchase Total per Person by Age Group

# Create my bins and give them labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
bin_names = ["<10", "10 - 14", "15 - 19", "20 - 24", "25 - 29", "30 - 34", "35 - 39","40+"]

# establish a new data fram based on the no duplicates data fram, add a column "Age Group", and bin everything by Age
# print the groupby object so I can view it
no_duplicates_binned = no_duplicates
no_duplicates_binned["Age Group"] = pd.cut(no_duplicates_binned["Age"], bins, labels=bin_names)
groupby_no_duplicates_binned = no_duplicates_binned.groupby(['Age Group'])
print(groupby_no_duplicates_binned)




C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [239]:
# Count by Age Group, Count the total players in no duplicates, and do a calculation on that object to get the %s
age_group_count = groupby_no_duplicates_binned["Age Group"].count()
total_players = no_duplicates["SN"].count()
age_group_perc = (age_group_count/total_players)*100


# Create a new data frame
purchase_by_age = pd.DataFrame({"Total Count": age_group_count,
                               "Percentage of Players": age_group_perc,
                               })

# format % of players using mapping
purchase_by_age["Percentage of Players"] = purchase_by_age["Percentage of Players"].map("{:.2f}%".format)

# Return the new data frame
purchase_by_age

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10 - 14,22,3.82%
15 - 19,107,18.58%
20 - 24,258,44.79%
25 - 29,77,13.37%
30 - 34,52,9.03%
35 - 39,31,5.38%
40+,12,2.08%


In [240]:
# Create bins for all instances of purchase data (includes duplicates of users)
all_instances_binned = purchase_data
all_instances_binned["Age Group"] = pd.cut(all_instances_binned["Age"], bins, labels=bin_names)
all_instances_groupby = all_instances_binned.groupby(['Age Group'])
print(all_instances_groupby)

# using count, mean, and sum of the groupby item and assigning those values to new variables
total_purchase_count = all_instances_groupby["SN"].count()
avg_purchase_price = all_instances_groupby["Price"].mean()
ttl_purchase_value = all_instances_groupby["Price"].sum()

# Use the variable age_group_count from above that only includes unique users
avg_total_per_person = ttl_purchase_value/age_group_count

# Create a new data frame for this data
purchase_detail_by_age = pd.DataFrame({"Purchase Count": total_purchase_count,
                               "Average Purchase Price": avg_purchase_price,
                               "Total Purchase Value": ttl_purchase_value,
                               "Avg Total Purchase Per Person": avg_total_per_person
                               })


# format $ values using mapping
purchase_detail_by_age["Average Purchase Price"] = purchase_detail_by_age["Average Purchase Price"].map("${:.2f}".format)
purchase_detail_by_age["Total Purchase Value"] = purchase_detail_by_age["Total Purchase Value"].map("${:.2f}".format)
purchase_detail_by_age["Avg Total Purchase Per Person"] = purchase_detail_by_age["Avg Total Purchase Per Person"].map("${:.2f}".format)

# Print the data frame
purchase_detail_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10 - 14,28,$2.96,$82.78,$3.76
15 - 19,136,$3.04,$412.89,$3.86
20 - 24,365,$3.05,$1114.06,$4.32
25 - 29,101,$2.90,$293.00,$3.81
30 - 34,73,$2.93,$214.00,$4.12
35 - 39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [241]:
### Top Spenders

# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#  * SN
#  * Purchase Count
#  * Average Purchase Price
#  * Total Purchase Value


# Create a groupby object using screennames
screenname_nodupes_groupby = purchase_data.groupby(["SN"])

# Do calculations for purchase count, avg purchase price, and total purchase value for each person
sn_purchase_count = screenname_nodupes_groupby["Price"].count()
sn_avg_purchase_price = screenname_nodupes_groupby["Price"].mean()
sn_total_purchase_value = screenname_nodupes_groupby["Price"].sum()

#Create a new data fram with this information
purchase_detail_by_sn = pd.DataFrame({"Purchase Count": sn_purchase_count,
                               "Average Purchase Price": sn_avg_purchase_price,
                               "Total Purchase Value": sn_total_purchase_value})


# Sort the new data frame descending from the highest total purchase value)
highest_purchase_value_df = purchase_detail_by_sn.sort_values("Total Purchase Value", ascending=False)

# format the data frame
highest_purchase_value_df["Average Purchase Price"] = highest_purchase_value_df["Average Purchase Price"].map("${:.2f}".format)
highest_purchase_value_df["Total Purchase Value"] = highest_purchase_value_df["Total Purchase Value"].map("${:.2f}".format)

# Print the data frame and show the top 5 users by total purchase value
highest_purchase_value_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [242]:
### Most Popular Items

# Identify the 5 most popular items by purchase count, then list (in a table):
#  * Item ID
#  * Item Name
#  * Purchase Count
#  * Item Price
#  * Total Purchase Value


# Create a groupby object indexed by item AND item name
item_nodupes_groupby = purchase_data.groupby(["Item ID", "Item Name"])

# Calculate the variables we are looking for
id_purchase_count = item_nodupes_groupby["Price"].count()
id_purchase_value = item_nodupes_groupby["Price"].sum()
id_purchase_price = id_purchase_value/id_purchase_count

# Create a new data frame
purchase_detail_by_item = pd.DataFrame({"Purchase Count": id_purchase_count,"Price": id_purchase_price, "Total Purchase Value": id_purchase_value})

# Sort the data fram
highest_purchase_item_df = purchase_detail_by_item.sort_values("Purchase Count", ascending=False)


# Format $s using mapping
highest_purchase_item_df["Price"] = highest_purchase_item_df["Price"].map("${:.2f}".format)
highest_purchase_item_df["Total Purchase Value"] = highest_purchase_item_df["Total Purchase Value"].map("${:.2f}".format)

# Print the top 5 items by purchase incidence
highest_purchase_item_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [246]:
### Most Profitable Items

# Identify the 5 most profitable items by total purchase value, then list (in a table):
#  * Item ID
#  * Item Name
#  * Purchase Count
#  * Item Price
#  * Total Purchase Value


# Re-sorting the previous data frame and storing in a new variable
highest_value_items = purchase_detail_by_item.sort_values("Total Purchase Value", ascending=False)

# Format using mapping
highest_value_items["Price"] = highest_value_items["Price"].map("${:.2f}".format)
highest_value_items["Total Purchase Value"] = highest_value_items["Total Purchase Value"].map("${:.2f}".format)

# Print the top 5 grossing items
highest_value_items.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
